In [1]:
!pip install "ray[rllib]" 

In [2]:
# Import the RL algorithm (Trainer) we would like to use.
from ray.rllib.agents.ppo import PPOTrainer

# Configure the algorithm.
config = {
    # Environment (RLlib understands openAI gym registered strings).
    "env": "Taxi-v3",
    # Use 2 environment workers (aka "rollout workers") that parallelly
    # collect samples from their own environment clone(s).
    "num_workers": 2,
    # Change this to "framework: torch", if you are using PyTorch.
    # Also, use "framework: tf2" for tf2.x eager execution.
    "framework": "tf",
    # Tweak the default model provided automatically by RLlib,
    # given the environment's observation- and action spaces.
    "model": {
        "fcnet_hiddens": [64, 64],
        "fcnet_activation": "relu",
    },
    # Set up a separate evaluation worker set for the
    # `trainer.evaluate()` call after training (see below).
    "evaluation_num_workers": 1,
    # Only for evaluation runs, render the env.
    "evaluation_config": {
        "render_env": True,
    },
}

# Create our RLlib Trainer.
trainer = PPOTrainer(config=config)

# Run it for n training iterations. A training iteration includes
# parallel sample collection by the environment workers as well as
# loss calculation on the collected batch and a model update.
for _ in range(3):
    print(trainer.train())

# Evaluate the trained Trainer (and render each timestep to the shell's
# output).
trainer.evaluate()


2022-06-28 10:06:51,212	WARNING trainer.py:2540 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Trainer.train()`. Instead, you will have to call `Trainer.evaluate()` manually in order to trigger an evaluation run.
2022-06-28 10:06:52,811	WARNING services.py:2002 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67096576 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=4.82gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2022-06-28 10:07:01,677	WARNING deprecation.py:46 -- DeprecationWarning: `simple_optimizer` has been deprecated. This will raise an error in the future!
2022-06-28 10:07:01,677	WARNING train

{'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'learner_stats': {'cur_kl_coeff': 0.20000000298023224, 'cur_lr': 4.999999873689376e-05, 'total_loss': 9.953198, 'policy_loss': 0.008731895, 'vf_loss': 9.944329, 'vf_explained_var': -8.963833e-06, 'kl': 0.0006804432, 'entropy': 1.7910774, 'entropy_coeff': 0.0, 'model': {}}, 'custom_metrics': {}, 'num_agent_steps_trained': 128.0}}, 'num_env_steps_sampled': 4000, 'num_env_steps_trained': 4000, 'num_agent_steps_sampled': 4000, 'num_agent_steps_trained': 4000}, 'sampler_results': {'episode_reward_max': -414.0, 'episode_reward_min': -983.0, 'episode_reward_mean': -783.1, 'episode_len_mean': 196.0, 'episode_media': {}, 'episodes_this_iter': 20, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [-929.0, -758.0, -830.0, -731.0, -785.0, -650.0, -812.0, -414.0, -749.0, -839.0, -794.0, -803.0, -821.0, -821.0, -785.0, -749.0, -722.0, -8

{'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'learner_stats': {'cur_kl_coeff': 0.10000000149011612, 'cur_lr': 4.999999873689376e-05, 'total_loss': 9.893939, 'policy_loss': 0.004841795, 'vf_loss': 9.888768, 'vf_explained_var': -0.00011243378, 'kl': 0.0033034717, 'entropy': 1.7833376, 'entropy_coeff': 0.0, 'model': {}}, 'custom_metrics': {}, 'num_agent_steps_trained': 128.0}}, 'num_env_steps_sampled': 8000, 'num_env_steps_trained': 8000, 'num_agent_steps_sampled': 8000, 'num_agent_steps_trained': 8000}, 'sampler_results': {'episode_reward_max': -123.0, 'episode_reward_min': -983.0, 'episode_reward_mean': -736.475, 'episode_len_mean': 193.55, 'episode_media': {}, 'episodes_this_iter': 20, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [-929.0, -758.0, -830.0, -731.0, -785.0, -650.0, -812.0, -414.0, -749.0, -839.0, -794.0, -803.0, -821.0, -821.0, -785.0, -749.0, -722.0

KeyboardInterrupt: 

In [ ]:
!rllib train --run APEX --env CartPole-v0

In [3]:
!rllib train --run DQN --env CartPole-v0

2022-06-28 10:07:17,776	WARNING services.py:2002 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67096576 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=4.03gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2022-06-28 10:07:19,496	ERROR syncer.py:147 -- Log sync requires rsync to be installed.
(DQNTrainer pid=2093) 2022-06-28 10:07:22,884	INFO trainer.py:2332 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
(DQNTrainer pid=2093) 2022-06-28 10:07:22,885	INFO simple_q.py:187 -- In multi-agent mode, policies wil

== Status ==
Current time: 2022-06-28 10:07:57 (running for 00:00:38.46)
Memory usage on this node: 8.6/15.6 GiB
Using FIFO scheduling algorithm.
Resources requested: 1.0/12 CPUs, 0/1 GPUs, 0.0/7.33 GiB heap, 0.0/3.66 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /root/ray_results/default
Number of trials: 1/1 (1 RUNNING)
+-----------------------------+----------+-----------------+--------+------------------+-------+----------+----------------------+----------------------+--------------------+
| Trial name                  | status   | loc             |   iter |   total time (s) |    ts |   reward |   episode_reward_max |   episode_reward_min |   episode_len_mean |
|-----------------------------+----------+-----------------+--------+------------------+-------+----------+----------------------+----------------------+--------------------|
| DQN_CartPole-v0_18300_00000 | RUNNING  | 172.17.0.2:2093 |     14 |          33.5751 | 14000 |   106.01 |                  200 |           